In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [3]:
torch.__version__

'2.1.0+cu118'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
train_df = pd.read_csv("drive/MyDrive/MNIST/train.csv")
test_df = pd.read_csv("drive/MyDrive/MNIST/test.csv")

X_val = train_df.iloc[:1000, 1:].values
X_train = train_df.iloc[1000:, 1:].values

y_val = train_df.iloc[:1000, 0].values
y_train = train_df.iloc[1000:, 0].values

X_test = test_df.values



In [70]:
X_val = torch.tensor(X_val, dtype = torch.float32)
X_train = torch.tensor(X_train, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train)
y_val = torch.tensor(y_val)

X_val /=255.0
X_train /=255.0
X_test /= 255.0


In [95]:
batch_size = 32

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size= batch_size)


In [96]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [106]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels=32, kernel_size=5)
    self.pool_2 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=3)
    self.pool_4 = nn.MaxPool2d(3)
    self.flat = nn.Flatten(start_dim=1, end_dim=-1)
    self.fc1 = nn.Linear(in_features=576, out_features=128)
    self.fc2 = nn.Linear(in_features=128, out_features=10)
    self.drop_out = nn.Dropout2d(0.2)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.conv1(x))
    x = self.drop_out(x)
    x = self.pool_2(x)
    x = self.relu(self.conv2(x))
    x = self.drop_out(x)
    x = self.pool_4(x)
    x = self.flat(x)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))

    return x


In [110]:
model = CNN().to(device)

In [111]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.2)

In [118]:
num_epochs = 10

for epoch in range(num_epochs):
  model.train()
  for batch_x, batch_y in train_loader:
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)
    optimizer.zero_grad()
    batch_x = batch_x.reshape(batch_x.shape[0], 1, 28, 28)
    output = model(batch_x)
    loss = criterion(output, batch_y)
    loss.backward()
    optimizer.step()

  model.eval()

  with torch.inference_mode():
    val_predict = []
    for batch_x, batch_y in val_loader:
      batch_x = batch_x.to(device)
      batch_y = batch_y.to(device)
      batch_x = batch_x.reshape(batch_x.shape[0], 1, 28, 28)
      output = model(batch_x)
      _, prediction = torch.max(output, 1)

      val_predict.extend(prediction.cpu().numpy())
    accuracy = accuracy_score(y_val, val_predict)
    print(f'Epoch[{epoch + 1}/{num_epochs}], Acc: {accuracy * 100:.2f}%')


Epoch[1/10], Acc: 99.00%
Epoch[2/10], Acc: 94.90%
Epoch[3/10], Acc: 98.80%
Epoch[4/10], Acc: 98.90%
Epoch[5/10], Acc: 99.00%
Epoch[6/10], Acc: 98.70%
Epoch[7/10], Acc: 99.20%
Epoch[8/10], Acc: 98.90%
Epoch[9/10], Acc: 98.90%
Epoch[10/10], Acc: 99.00%


In [115]:
with torch.inference_mode():
  X_test = X_test.to(device)
  X = X_test.reshape(X_test.shape[0], 1, 28, 28)
  output = model(X)
  _, predictions = torch.max(output, axis = 1)

print(predictions)



tensor([2, 0, 9,  ..., 3, 9, 2], device='cuda:0')


In [117]:
ImageId = np.arange(predictions.shape[0]) + 1
submission = pd.DataFrame({'ImageId': ImageId, 'Label': predictions.cpu()})
submission.to_csv('submission.csv', index=False)
